In [ ]:
import os
import numpy as np 
import pickle 
import matplotlib.pyplot as plt 
from collections import OrderedDict
np.set_printoptions(suppress=True)

In [ ]:
dset = 'eth' # 'eth', 'zara1','hotel'

In [ ]:
#Trajectorn++
trajpp = np.load('./raw/Trajectron++/' + dset + '_test_traj++.npz')
trajpp_other = np.load('./raw/Trajectron++/' + dset + '_test_traj_gmm.npz')

trajpp_ = {}
trajpp_['obs'] = np.transpose(trajpp_other['obs'], axes=(0, 2, 1))
trajpp_['tarj'] = np.transpose(trajpp['gt_abs'], axes=(0, 2, 1))
trajpp_['tarj'] = trajpp_['tarj'] - trajpp_['obs'][..., 0:1]
trajpp_['tarj_ref'] = np.transpose(trajpp['gt_rel'], axes=(0, 2, 1))
trajpp_['pred'] = np.transpose(trajpp['pred_abs'], axes=(
    0, 1, 3, 2))[:, :100, ...] - trajpp_['obs'][:, None, :, 0:1]
trajpp_['obs'] = trajpp_['obs'] - trajpp_['obs'][..., 0:1]

len(trajpp_['obs'])

In [ ]:
with open('./raw/Social-STGCNN/save_' + dset + '.pkl', 'rb') as f:
    sstgcnn = pickle.load(f)

sstgcnn_ = {}
sstgcnn_['obs'] = np.concatenate(sstgcnn['obs'], axis=1).squeeze()
sstgcnn_['tarj'] = np.concatenate(sstgcnn['tarj'], axis=1).squeeze()
sstgcnn_['tarj'] = sstgcnn_['tarj'] - sstgcnn_['obs'][..., 0:1]
full_traj = np.concatenate((sstgcnn_['obs'], sstgcnn_['tarj']), axis=-1)
sstgcnn_['tarj_ref'] = (full_traj[..., 1:] - full_traj[..., :-1])[..., -12:]
PRED = []
for i in range(len(sstgcnn['pred'])):
    PRED.append(np.stack(sstgcnn['pred'][i]))
sstgcnn_['pred'] = np.transpose(
    np.concatenate(PRED, axis=2).squeeze(),
    axes=(2, 0, 3, 1)) - sstgcnn_['obs'][:, None, :, 0:1]
sstgcnn_['obs'] = sstgcnn_['obs'] - sstgcnn_['obs'][..., 0:1]
len(sstgcnn_['obs'])

In [ ]:
with open('./raw/Social-Implicit/save_' + dset + '.pkl', 'rb') as f:
    simplicit = pickle.load(f)

simplicit_ = {}
simplicit_['obs'] = np.concatenate(simplicit['obs'], axis=1).squeeze()
simplicit_['tarj'] = np.concatenate(simplicit['tarj'], axis=1).squeeze()
simplicit_['tarj'] = simplicit_['tarj'] - simplicit_['obs'][..., 0:1]
full_traj = np.concatenate((simplicit_['obs'], simplicit_['tarj']), axis=-1)
simplicit_['tarj_ref'] = (full_traj[..., 1:] - full_traj[..., :-1])[..., -12:]
PRED = []
for i in range(len(simplicit['pred'])):
    PRED.append(np.stack(simplicit['pred'][i]))
simplicit_['pred'] = np.transpose(
    np.concatenate(PRED, axis=2).squeeze(),
    axes=(2, 0, 3, 1)) - simplicit_['obs'][:, None, :, 0:1]
simplicit_['obs'] = simplicit_['obs'] - simplicit_['obs'][..., 0:1]

len(simplicit_['obs'])

In [ ]:
with open('./raw/ExpertTraj/save_' + dset + '.pkl', 'rb') as f:
    expertraj = pickle.load(f)

expertraj_ = {}
expertraj_['obs'] = np.transpose(np.concatenate(expertraj['obs'],
                                                axis=1).squeeze(),
                                 axes=(1, 2, 0))
expertraj_['tarj'] = np.transpose(np.concatenate(expertraj['tarj'],
                                                 axis=1).squeeze(),
                                  axes=(1, 2, 0))
full_traj = np.concatenate((expertraj_['obs'], expertraj_['tarj']), axis=-1)
expertraj_['tarj_ref'] = (full_traj[..., 1:] - full_traj[..., :-1])[..., -12:]
PRED = []
for i in range(len(expertraj['pred'])):
    PRED.append(np.stack(expertraj['pred'][i]))
expertraj_['pred'] = np.transpose(np.concatenate(PRED, axis=2).squeeze(),
                                  axes=(2, 0, 3, 1))

len(expertraj_['obs'])

In [ ]:
with open('./raw/Social-GAN/save_'+dset+'.pkl','rb') as f:
    sgan = pickle.load(f)

sgan_ = {}
sgan_['obs'] = np.transpose(np.concatenate(sgan['obs'],axis = 1).squeeze(), axes=(1,2,0))
sgan_['tarj'] = np.transpose(np.concatenate(sgan['tarj'],axis = 1).squeeze(), axes=(1,2,0))
sgan_['tarj'] = sgan_['tarj']-sgan_['obs'][...,0:1]
full_traj =np.concatenate((sgan_['obs'], sgan_['tarj']),axis=-1)
sgan_['tarj_ref'] = (full_traj[...,1:]- full_traj[...,:-1])[...,-12:]
PRED = []
for i in range(len(sgan['pred'])):
    PRED.append(np.stack(sgan['pred'][i]))
sgan_['pred'] = np.transpose(np.concatenate(PRED,axis = 2).squeeze(), axes=(2,0,3,1))- sgan_['obs'][:, None, :, 0:1]
sgan_['obs'] = sgan_['obs']- sgan_['obs'][...,0:1]

len(sgan_['tarj'])

In [ ]:
def get_closer_index(key,query):
    diff = np.linalg.norm(query - key, axis=-2).sum(-1)
    argmin = np.argmin(diff,axis=0)
    # print(diff[argmin], argmin, "\n",key,"\n",query[argmin])
    return argmin

In [ ]:
collect = []

for key in  range(len(sstgcnn_['obs'])):
    collect.append(OrderedDict())
    key_value = sstgcnn_['obs'][key]

    key_trajpp = get_closer_index(key_value, trajpp_['obs'])
    key_sgan = get_closer_index(key_value, sgan_['obs'])
    key_simplicit = get_closer_index(key_value, simplicit_['obs'])
    key_expertraj = get_closer_index(key_value, expertraj_['obs'])

    collect[-1]['Observation'] = sstgcnn_['obs'][key]
    collect[-1]['GroundTruth'] = sstgcnn_['tarj'][key]
    collect[-1]['S-GAN'] = sgan_['pred'][key_sgan]
    collect[-1]['S-STGCNN'] = sstgcnn_['pred'][key]
    collect[-1]['S-Implicit'] = simplicit_['pred'][key_simplicit]
    collect[-1]['Trajectron++'] = trajpp_['pred'][key_trajpp]
    collect[-1]['ExpertTraj'] = expertraj_['pred'][key_expertraj]


In [ ]:
# print stats 
len(collect)

In [ ]:
if not os.path.exists('./aligned'):
    os.makedirs('./aligned')

In [ ]:
with open('./aligned/vis_'+dset+'.pkl','wb') as f : 
    pickle.dump(collect,f)